this notebook try to summerize useful command in minimal way

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt


# load data

In [ ]:
df = pd.read_csv('') # datapath 

In [ ]:
df.head() # see data sample

In [ ]:
df.describe() # see over all data

In [ ]:
df.info() # see data type

# EDA

note: for classification problem you should always use hua = 'y' which 'y' is label name

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(data=df, x="catagory data") # see the number of each class

In [ ]:
sns.scatterplot(data=df, x="continuous_data", y="continuous_data",hue='y') # see the relation between feature

In [ ]:
sns.histplot(data=df, x="continuous_data") # see the distribution 

In [ ]:
sns.boxplot(data=df, x="catagory_data or y", y="continuous_data") # see the distibution over catagory data

In [ ]:
sns.barplot(data = df ,x = 'y', y = "continuous_data", hue="y") # 

easy for any task

In [ ]:
sns.pairplot(df, hue = 'y') # see the distribution and scaterplot and also can do 

In [ ]:
sns.heatmap(df.corr(), annot=True) # see the correation

In [ ]:
import ppscore as pps

# before using pps, let's drop country and year
dfcopy = df.copy()
dfcopy.drop(['catagory_data1','catagory_data2'], axis='columns', inplace=True)

#this needs some minor preprocessing because seaborn.heatmap unfortunately does not accept tidy data
matrix_df = pps.matrix(dfcopy)[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')

#plot
plt.figure(figsize = (15,8))
sns.heatmap(matrix_df, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)

have you ever try this?

In [ ]:
import sweetviz
report = sweetviz.analyze([df,'train'],target_feat='target_name')
report.show_html('report.html')

# data engineering

In [ ]:
# change the meaning of data from what you get from eda ex cartesian to polar, take log, different value, grouping ..

In [ ]:
# plot again to see new feature

# split data

In [ ]:
# data splite
X = df[['feature1','feature1']]
y = df["y"]
# use all feature 

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 96, stratify=y)
# used satify to split balancly on y

# preprocess

## imputation

In [ ]:
X_train.isna().sum() # check null value

In [ ]:
y_train.value_counts() # check catagory value

## encoding

## outlier

In [ ]:
# see the outlier percentage
def outlier_count(col, data = X_train):
    
    # calculate your 25% quatile and 75% quatile
    q75, q25 = np.percentile(data[col], [75, 25])
    
    # calculate your inter quatile
    iqr = q75 - q25
    
    # min_val and max_val
    min_val = q25 - (iqr*1.5)
    max_val = q75 + (iqr*1.5)
    
    # count number of outliers, which are the data that are less than min_val or more than max_val calculated above
    outlier_count = len(np.where((data[col] > max_val) | (data[col] < min_val))[0])
    
    # calculate the percentage of the outliers
    outlier_percent = round(outlier_count/len(data[col])*100, 2)
    
    if(outlier_count > 0):
        print("\n"+15*'-' + col + 15*'-'+"\n")
        print('Number of outliers: {}'.format(outlier_count))
        print('Percent of data that is outlier: {}%'.format(outlier_percent))

## scalling

In [ ]:
from sklearn.preprocessing import StandardScaler

# feature scaling helps improve reach convergence faster
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# training

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
# use x1
#any random_state you can use......up to you 
lr = LogisticRegression(random_state=96)
rf = RandomForestClassifier(random_state=96)
sv = SVC(random_state=96)
gnb = GaussianNB()

models = [lr, rf, sv,gnb]

#3.2 perform cross validation using KFold
from sklearn.model_selection import KFold, cross_val_score

kfold = KFold(n_splits = 5, shuffle = True, random_state=999)

for model in models:
    score = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')  #f1, recall, precision, accuracy
    print("Scores: ", score, "- Scores mean: ", score.mean(), "- Scores std (lower better): ", score.std())  #out of 1 ; 1 means perfect accuracy
    #lr, rf, sv

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

# Offline upsampling could look like this:
#first upsample / downsample
smote = SMOTE(random_state = 11)
X_train, y_train = smote.fit_resample(X_train, y_train)

#then do scaling + modeling
pipeline = Pipeline(steps = [['scaler', MinMaxScaler()], ['classifier', LogisticRegression(random_state=11, max_iter=1000)]])

param_grid = {'C': [1, 10], 'kernel': ('linear', 'rbf')}
kfold = KFold(n_splits = 5, shuffle = True, random_state=999) 
grid_search = GridSearchCV(estimator=pipeline,  param_grid=param_grid, scoring='roc_auc', cv=kfold,  n_jobs=-1)

# Online upsampling could look like this:

#put smote as pipeline
pipeline = Pipeline(steps = [['smote', SMOTE(random_state=11)], ['scaler', MinMaxScaler()], ['classifier', LogisticRegression(random_state=11,  max_iter=1000)]])

param_grid = {'C': [1, 10], 'kernel': ('linear', 'rbf')}
kfold = KFold(n_splits = 5, shuffle = True, random_state=999)
grid_search = GridSearchCV(estimator=pipeline,  param_grid=param_grid, scoring='roc_auc', cv=kfold,  n_jobs=-1)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'bootstrap': [True], 'max_depth': [5, 10, None],
              'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

rf = RandomForestRegressor(random_state = 1)

grid = GridSearchCV(estimator = rf, 
                    param_grid = param_grid, 
                    cv = kfold, 
                    n_jobs = -1, 
                    return_train_score=True, 
                    refit=True,
                    scoring='neg_mean_squared_error')

# Fit your grid_search
grid.fit(X_train, y_train);  #fit means start looping all the possible parameters

In [ ]:
grid.best_score_

In [ ]:
dt = grid.best_estimator_

dt.feature_importances_

# Test

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test,model.predict(X_test)))